In [2]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

<IB connected to 127.0.0.1:7497 clientId=15>

In [3]:
# from ib_insync import *
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import math
import datetime 
import statsmodels.formula.api as smf 
from datetime import date, time, datetime, timedelta
import seaborn as sns
import random
from ta import add_all_ta_features
from ta.utils import dropna
from ta.trend import *
from collections import deque, defaultdict
import helpers 

In [4]:
### Initialization of hard coded data ###

order_books = defaultdict(set)
df_ratio = pd.read_csv("ratio_dict.csv", index_col = 0).set_index("Pair")
df_global = pd.DataFrame({"date":[]})

aus_adr_underlying_pairs = [
    ["ATHE", "ATH"],["GENE", "GTG"],["IMMP", "IMM"],["IMRN","IMC"],["JHX", "JHX"],["KZIA", "KZA"],
    ["MESO","MSB"],["PLL", "PLL"],["WBK","WBC"]
]

hk_adr_underlying_pairs = [
    ["ACH", "2600"],["BGNE", "6160"],["CEA","670"],["HNP", "902"],["LFC", "2628"],["PTR", "857"],
    ["SHI", "338"],["SNP","386"],["ZNH", "1055"]
]

jpn_adr_underlying_pairs = [
    ["CAJ", "7751"],["HMC","7267"],["IX","8591"],["MFG","8411"],["MUFG","8306"],["NMR","8604"],
    ["SMFG","8316"],["SONY","6758"],["TAK", "4502"],["TM", "7203"]
]

country_info = {
    "Australia":{"Currency": "AUD", 
                "Exchange": "SMART",
                "Pairs": aus_adr_underlying_pairs},
    
    "Japan":{"Currency": "JPY", 
             "Exchange": "TSEJ",
             "Pairs": jpn_adr_underlying_pairs},
    
    "HK":{"Currency": "HKD", 
         "Exchange": "SEHK",
         "Pairs": hk_adr_underlying_pairs},
    
}

forex_pairs = [("AUD", "USD"),  ("USD", "HKD"), ("USD","JPY")]


# functions return the market opening time adjusted for DST

us_dst = {
        2021: (datetime(2021, 3, 14), datetime(2021, 11, 7)),
        2020: (datetime(2020, 3, 8),  datetime(2020, 11, 1)),
        2019: (datetime(2019, 3, 10), datetime(2019, 11, 3)),
        2018: (datetime(2018, 3, 11), datetime(2018, 11, 4)),
        2017: (datetime(2017, 3, 12), datetime(2017, 11, 5)),
        2016: (datetime(2016, 3, 13), datetime(2016, 11, 6)),
        2015: (datetime(2015, 3, 9),  datetime(2015, 11, 2)),
    } 

def next_weekday(date_now):
    if date_now.weekday()<4:
        return date_now + timedelta(days = 1)
    
    while date_now.weekday()>3:
        date_now += timedelta(days = 1)
        
    return date_now

def next_market_open_day(date_now):
    '''
    Returns today if Mon - Thurs, returns next Mon if Fri thru Sun
    '''
    while date_now.weekday()>4:
        date_now += timedelta(days = 1)
    return date_now

def AUS_opening(date_now):
    date_adj = next_market_open_day(date_now)
    us_start, us_end = us_dst[date_adj.year]
    if us_start < date_adj and date_adj < us_end:
        return datetime.combine(date_adj,time(10,0)) - timedelta(hours = 14)
    else:
        return datetime.combine(date_adj, time(10,0)) - timedelta(hours = 15)

def AUS_closing(date_now):
    open_time = AUS_opening(date_now)
    return open_time + timedelta(hours = 6)

def HK_opening(date_now):
    date_adj = next_market_open_day(date_now)
    start, end = us_dst[date_adj.year]
    if start < date_adj and date_adj < end:
        return datetime.combine(date_adj,time(9,30)) - timedelta(hours = 12)
    else:
        return datetime.combine(date_adj, time(9,30)) - timedelta(hours = 13)
    
def HK_closing(date_now):
    open_time = HK_opening(date_now)
    return open_time + timedelta(hours = 6.5)
    
def JPN_opening(date_now):
    date_now = next_market_open_day(date_now)
    start, end = us_dst[date_now.year]
    if start < date_now and date_now < end:
        return datetime.combine(date_now,time(9,0)) - timedelta(hours = 13)
    else:
        return datetime.combine(date_now,time(9,0)) - timedelta(hours = 14)
    
def JPN_closing(date_now):
    open_time = JPN_opening(date_now)
    return open_time + timedelta(hours = 6)

def US_opening(date_now):
    return datetime.combine(next_market_open_day(date_now),time(9,30))

def US_closing(date_now):
    return datetime.combine(next_market_open_day(date_now),time(16,0))

##### Initialization of global variables here ###

order_books = defaultdict(set)
df_ratio = pd.read_csv("ratio_dict.csv", index_col = 0).set_index("Pair")
df_global = pd.DataFrame({"date":[]})


country_info = {
    "Australia":{"Currency": "AUD", 
                "Exchange": "SMART",
                "Pairs": aus_adr_underlying_pairs},
    
    "Japan":{"Currency": "JPY", 
             "Exchange": "TSEJ",
             "Pairs": jpn_adr_underlying_pairs},
    
    "China":{"Currency": "HKD", 
         "Exchange": "SEHK",
         "Pairs": hk_adr_underlying_pairs},
    
}

In [5]:
# initialize data lists, hyperparameters etc

path_data = "live_trading_data/"
df_forex = pd.DataFrame({"date":[]})
def initialize_forex():
    global df_forex, country_info

    now = datetime.now()
    df_forex = pd.read_csv(f"{path_data}df_forex.csv")
    df_forex["date"] = [datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S') for date_time in df_forex["date"]]
    pull_head = datetime.combine(max(now-timedelta(days = round(7/5*100)),df_forex["date"].iloc[-1]).date(),time(17,0))


    now = datetime.combine(datetime.now().date(), time(17,5))
    df_temp_jpy = pd.DataFrame({"date":[]})
    df_temp_hkd = pd.DataFrame({"date":[]})
    df_temp_aud = pd.DataFrame({"date":[]})
    df_temp_holder = {
        "AUD.USD" : df_temp_aud,
        "JPY.USD" : df_temp_jpy,
        "HKD.USD" : df_temp_hkd
    }
    pull_head = datetime(2021,4,15)
    while pull_head < now:
        print(pull_head.date())
        for currency1, currency2 in forex_pairs:

            history = "2 D"
            freq = "1 min"
            side = "BID_ASK"

            forex_contract = Contract(symbol = currency1, secType = "CASH", exchange = "IDEALPRO", currency = currency2)
            ib.qualifyContracts(forex_contract)
            try:
                df_temp = get_data(forex_contract, history, freq, side, pull_head)    

                if currency2 == "USD":
                    df_temp[f"{currency1}.{currency2}"] = (df_temp["open"] + df_temp["close"])/2
                    forex_string = f"{currency1}.{currency2}"
                else:
                    df_temp[f"{currency2}.{currency1}"] = 2/(df_temp["open"] + df_temp["close"])
                    forex_string = f"{currency2}.{currency1}"

                df_temp["date"] = df_temp["date"].astype('datetime64[s]')
    #             if pull_head == datetime(2021,4,21): print(df_temp)

                df_temp_holder[forex_string] = pd.merge(df_temp_holder[forex_string], df_temp[["date",forex_string]], how = "outer")
            except Exception as e:
                print(e)

        pull_head += timedelta(days = 1)
    df_temp_2 = pd.merge(df_temp_holder["HKD.USD"],pd.merge(df_temp_holder["AUD.USD"], df_temp_holder["JPY.USD"],  how = "outer"), on = "date", how = "outer")
    df_forex = pd.merge(df_forex,df_temp_2, how = "outer").drop_duplicates(keep = "first")
    #     df_forex.to_csv(f"{path_data}df_forex.csv", index = False)
    print("df_forex initialized")

def save_forex():
    global df_forex
    df_forex.drop_duplicates("date")
    df_forex.to_csv(f"{path_data}df_forex.csv", index = False)

In [6]:
country_info = {
    "Australia":{"Currency": "AUD", 
                "Exchange": "SMART",
                "Pairs": aus_adr_underlying_pairs},
    
    "Japan":{"Currency": "JPY", 
             "Exchange": "TSEJ",
             "Pairs": jpn_adr_underlying_pairs},
    
    "HK":{"Currency": "HKD", 
         "Exchange": "SEHK",
         "Pairs": hk_adr_underlying_pairs},
    
}


# Store all the historical adr/underlying open and close data

#define a function to get IB data. endDate is the last date of the historical data 

def get_data(contract, history, freq, side, endDate =''):
    bar = ib.reqHistoricalData(
        contract,
        endDateTime = endDate,
        durationStr=history,
        barSizeSetting= freq,
        whatToShow=side,
        useRTH=True,
        formatDate=1)
    return util.df(bar)

def pull_asian_close(days = 1):
    global df_global, country_info

    today = datetime.now().date()
    history = f"{days} D"
    freq = "1 Day"
    side = 'Trades'
    
    df_asian_close = pd.DataFrame({"date":[]})
    for key in country_info.keys():
        pairs = country_info[key]["Pairs"]
        exchange = country_info[key]["Exchange"]
        currency = country_info[key]["Currency"]
        
        for p in pairs:
            underlying = p[1]
            contract = Contract(symbol = underlying, secType = 'STK', exchange = exchange, currency = currency)

            df = get_data(contract, history, freq, side, endDate =today)
            ratio = max(1, df_ratio.loc[f"{p[0]}_{p[1]}"]["stock_num_per_adr"])
            df[f"underlying_{underlying}_close_per_unit"] = df["close"] * ratio
            df = df.loc[:, ["date", "close", f"underlying_{underlying}_close_per_unit"]]
            col_name = "underlying_" + underlying + "_close"
            df = df.rename(columns = {"close":col_name})
            df_asian_close = pd.merge(df, df_asian_close, how = "outer")

            
    df_global = pd.merge(df_global, df_asian_close,how = "outer").sort_values("date")

    
    
def pull_us_close(days = 1):
    global df_global, country_info

    today = datetime.now().date()
    history = f"{days} D"
    freq = "1 Day"
    side = 'Trades'
    
    df_us_close = pd.DataFrame({"date":[]})
    for key in country_info.keys():
        pairs = country_info[key]["Pairs"]
        exchange = "SMART"
        currency = "USD"
        
        for p in pairs:
            underlying = p[0] # get the ticker of the ADR
            contract = Contract(symbol = underlying, secType = 'STK', exchange = exchange, currency = currency)
            df = get_data(contract, history, freq, side, endDate ='')
            ratio = max(1, df_ratio.loc[f"{p[0]}_{p[1]}"]["adr_num_per_stock"])
            df[f"adr_{p[0]}_close_per_unit"] = df["close"] * ratio
            df = df.loc[:, ["date", "close", f"adr_{p[0]}_close_per_unit" ]]
            col_name = "adr_" + underlying + "_close"
            df = df.rename(columns = {"close":col_name})
            df_us_close = pd.merge(df, df_us_close, how = 'outer')
            
    df_global = pd.merge(df_global, df_us_close,how = 'outer').sort_values("date")
    
def update_df_global_with_forex():
    global df_global, df_forex
    
    us_opens = [datetime.combine(x, time(9,30)) - timedelta(minutes = 5) for x in df_global["date"]]
    forex_before_us_opens = df_forex[df_forex["date"].isin(us_opens)].drop_duplicates(["date"]).sort_values(["date"]).reset_index(drop = True)
    forex_before_us_opens["date"] = [datetime.date(x) for x in forex_before_us_opens["date"]]
    forex_before_us_opens.columns = ["date", "JPY.USD_before_us_open", "AUD.USD_before_us_open", "HKD.USD_before_us_open"]

    # Add extra date
    dates = list(df_global["date"])
    last_date = dates[-1]
    dates.append(next_weekday(last_date))
    aus_opens = [AUS_opening(datetime.combine(x, time(0,0))) - timedelta(minutes = 5) for x in dates]
    hk_opens = [HK_opening(datetime.combine(x, time(0,0))) - timedelta(minutes = 5) for x in dates]
    jap_opens = [JPN_opening(datetime.combine(x, time(0,0))) - timedelta(minutes = 5) for x in dates]
    aus_forex_opens =  df_forex[df_forex["date"].isin(aus_opens)].drop_duplicates(["date"]).sort_values(["date"]).reset_index(drop = True)
    hk_forex_opens =  df_forex[df_forex["date"].isin(hk_opens)].drop_duplicates(["date"]).sort_values(["date"]).reset_index(drop = True)
    jap_forex_opens =  df_forex[df_forex["date"].isin(jap_opens)].drop_duplicates(["date"]).sort_values(["date"]).reset_index(drop = True)
    aus_forex_opens["date"] += timedelta(days = 1)
    hk_forex_opens["date"] += timedelta(days = 1)
    jap_forex_opens["date"] += timedelta(days = 1)
    aus_forex_opens["date"] = [datetime.date(x) for x in aus_forex_opens["date"]]
    hk_forex_opens["date"] = [datetime.date(x) for x in hk_forex_opens["date"]]
    jap_forex_opens["date"] = [datetime.date(x) for x in jap_forex_opens["date"]]
    forex_before_asian_opens = pd.merge(pd.merge(aus_forex_opens[["date", "AUD.USD"]], jap_forex_opens[["date", "JPY.USD"]], how = "outer"),
                                        hk_forex_opens[["date", "HKD.USD"]], how = "outer")
    forex_before_asian_opens.columns = ["date", "JPY.USD_before_asian_open", "AUD.USD_before_asian_open", "HKD.USD_before_asian_open"]

    df_global = pd.merge(df_global, 
                         pd.merge(forex_before_us_opens, forex_before_asian_opens, how = "outer"),
                         how = "outer").sort_values("date")

def pull_forex_data(history = "1 D", freq = "1 min", side = "BID_ASK", end_date = "" ):
    '''
    Updates all three forex pairs
    '''
    
    global df_global, country_info, forex_pairs, test, df_forex

    df_temp = pd.DataFrame({"date":[]})
    for currency1, currency2 in forex_pairs:
        contract = Contract(symbol = currency1, secType = 'CASH', exchange = "IDEALPRO", currency = currency2)
        df = get_data(contract, history, freq, side)

        if currency2 == "USD":
            df[f"{currency1}.{currency2}"] = (df["open"] + df["close"])/2
            forex_string = f"{currency1}.{currency2}"
        else:
            df[f"{currency2}.{currency1}"] = 2/(df["open"] + df["close"])
            forex_string = f"{currency2}.{currency1}"
        df_temp = pd.merge(df_temp, df[['date',forex_string]], how = "outer")
    
    df_forex = pd.merge(df_forex, df_temp, how = "outer")
    update_df_global_with_forex()
    
    
def convert_price_asian_close():
    
    """
    Convert the last Asian prices we pulled before US open into US dollars
    Using the most recent forex data
    """
    global df_global, df_forex, df_ratio, country_info
    
    # We use the last 
    for key in country_info:
        pairs = country_info[key]["Pairs"]
        for pair in pairs:
            underlying_col_name = f"underlying_{pair[1]}_close"
            pair_name =  pair[0] + "_" + pair[1]
            ratio = df_ratio.loc[pair_name,"adr_num_per_stock"] # Determing number of adr per stock
            
            forex_col_name = country_info[key]["Currency"]+".USD"
            fx = df_forex.loc[len(df_forex)-1, forex_col_name]
            price_in_usd = df_global.loc[len(df_global)-1, underlying_col_name]*ratio*fx
            
            df_global.at[len(df_global)-1, underlying_col_name] = price_in_usd

In [8]:
initialize_forex()
pull_asian_close(100)
pull_us_close(100)
pull_forex_data()
#convert_price_asian_close()

In [9]:
df_global.head(10)

,date,underlying_1055_close,underlying_1055_close_per_unit,underlying_386_close,underlying_386_close_per_unit,underlying_338_close,underlying_338_close_per_unit,underlying_857_close,underlying_857_close_per_unit,underlying_2628_close,...,adr_GENE_close,adr_GENE_close_per_unit,adr_ATHE_close,adr_ATHE_close_per_unit,JPY.USD_before_us_open,AUD.USD_before_us_open,HKD.USD_before_us_open,JPY.USD_before_asian_open,AUD.USD_before_asian_open,HKD.USD_before_asian_open
0,2020-11-26,4.84,242.0,3.83,383.0,1.67,167.0,2.65,265.0,17.96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-11-27,4.84,242.0,3.77,377.0,1.63,163.0,2.63,263.0,18.50,...,3.18,3.18,1.550,1.550,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-11-30,4.76,238.0,3.51,351.0,1.72,172.0,2.47,247.0,17.58,...,3.17,3.17,1.520,1.520,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-12-01,4.93,246.5,3.56,356.0,1.78,178.0,2.45,245.0,18.02,...,3.30,3.30,1.400,1.400,0.009579,0.735475,0.128997,0.735935,0.009587,0.129004
4,2020-12-02,4.91,245.5,3.63,363.0,1.71,171.0,2.47,247.0,18.02,...,3.15,3.15,1.440,1.440,0.009565,0.735550,0.129000,0.738080,0.009586,0.128996
5,2020-12-03,5.14,257.0,3.63,363.0,1.72,172.0,2.48,248.0,17.90,...,3.11,3.11,1.480,1.480,0.009625,0.744040,0.129011,0.741335,0.009573,0.128999
6,2020-12-04,5.12,256.0,3.54,354.0,1.69,169.0,2.45,245.0,17.78,...,3.15,3.15,1.480,1.480,0.009621,0.743020,0.129023,0.743820,0.009633,0.129009
7,2020-12-07,5.00,250.0,3.57,357.0,1.68,168.0,2.40,240.0,17.60,...,3.26,3.26,1.420,1.420,0.009607,0.742495,0.129022,0.743215,0.009596,0.129021
8,2020-12-08,4.92,246.0,3.40,340.0,1.61,161.0,2.36,236.0,17.38,...,3.41,3.41,1.490,1.490,0.009611,0.741580,0.129018,0.741870,0.009615,0.129017
9,2020-12-09,4.87,243.5,3.36,336.0,1.58,158.0,2.35,235.0,17.36,...,4.47,4.47,1.425,1.425,0.009598,0.747365,0.128997,0.740790,0.009602,0.129010


Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
[WinError 10054] An existing connection was forcibly closed by the remote host


In [245]:
def place_order(country, security, sectype, side, quantity, price):
    global order_books
    exchange = country_info[country]["Exchange"]
    currenct = country_info[country]["Currency"]
    
    # Define order 
    contract = Contract(symbol = security, secType = 'STK', exchange =exchange, currency = currency)
    order = LimitOrder(side,quantity,price)
    
    # Place the order
    msg = ib.placeOrder(contract, order)
    orderId = msg.order.orderId
    
    # Add the orderId to orderbook
    name =  security +"_"+ sectype
    order_books[name].add(orderId)
    
    

def check_open_order(security, sectype = "underlying"):
    """
    Check if we have open order for this particular security
    Args: 
    security (str) name of security
    sectype(str): takes th evalue either "adr" or "underlying"
    
    return: the quantity of open orders (orders that we wanted to trade but not traded) of a particular security
    
    """
    global order_books
    ct = 0
    lookup = security +"_"+ sectype
    all_orders = order_books[lookup]
    open_orders = ib.openOrders()
    for o in openOders:
        if o.orderId in all_orders:
            ct += o.totalQuantity
        
    return ct
        

In [209]:
# Pairs we have chosen to trade
trading_limits = {"Australia" : 4, "China" : 1, "Japan" : 3}
allocation = {"Australia" : 0.3, "China" : 0.4, "Japan" : 0.3}

list_pairs = [("Australia", "PLL_PLL"),
               ("Australia", "MESO_MSB"),
               ("Australia", "GENE_GTG"),
               ("Australia", "WBK_WBC"),
               ("Australia", "KZIA_KZA"),
               ("Australia", "IMMP_IMM"),
               ("Australia", "IMRN_IMC"),
               ("Australia", "ATHE_ATH"),
               ("Australia", "JHX_JHX"),
               ("Japan", "SONY_6758"), 
               ("Japan", "TAK_4502"),
               ("Japan", "TM_7203"),
               ("Japan", "SMFG_8316"),
               ("China", "BGNE_6160"), 
               ("China", "SNP_386")]


fname = 'results1_sfx_all.txt'
with open(fname, 'r') as f:
    is_res = f.readlines()
hp = {}
for i in range(len(list_pairs)):
    hp[list_pairs[i]] = [float(x) for x in is_res[i*5 + 4].split("(")[1].split(")")[0].split(", ")]
hp_dict = {}
for (country, adr) in list_pairs:
    hp_dict[(country, adr)] = {}
    hp_dict[(country, adr)]["lookback"] = int(hp[(country, adr)][0])
    hp_dict[(country, adr)]["entry"] = hp[(country, adr)][1]
    hp_dict[(country, adr)]["exit"] = hp[(country, adr)][2]
    hp_dict[(country, adr)]["stop_loss"] = hp[(country, adr)][3]
    # Fraction of cash allocated to each adr-stock pair
    hp_dict[(country, adr)]["allocation"] = allocation[country]/trading_limits[country]
    hp_dict[(country, adr)]["original_allocation"] = allocation[country]/trading_limits[country]

diff_record_dict = {}
for (country, adr) in list_pairs:
    diff_record_dict[(country, adr)] = deque(maxlen = 2*hp_dict[(country, adr)]["lookback"])
    
conditions = {}
    for (country, adr) in list_pairs:
        conditions[(country, adr)] = {}
        conditions[(country, adr)]["enter_cond1"] = False
        conditions[(country, adr)]["exit_cond1"] = False
        conditions[(country, adr)]["enter_cond2"] = False
        conditions[(country, adr)]["exit_cond2"] = False

    
positions = {}
for (country, adr) in list_pairs:
    positions[(country, adr)] = {}
    positions[(country, adr)]["stock_pos"] = 0
    positions[(country, adr)]["adr_pos"] = 0
    positions[(country, adr)]["prev_adr_pos"] = 0
    positions[(country, adr)]["holding_period"] = None
    positions[(country, adr)]["trade_type"] = None


{('Australia', 'PLL_PLL'): deque([], maxlen=200), ('Australia', 'MESO_MSB'): deque([], maxlen=200), ('Australia', 'GENE_GTG'): deque([], maxlen=200), ('Australia', 'WBK_WBC'): deque([], maxlen=120), ('Australia', 'KZIA_KZA'): deque([], maxlen=120), ('Australia', 'IMMP_IMM'): deque([], maxlen=200), ('Australia', 'IMRN_IMC'): deque([], maxlen=60), ('Australia', 'ATHE_ATH'): deque([], maxlen=120), ('Australia', 'JHX_JHX'): deque([], maxlen=200), ('Japan', 'SONY_6758'): deque([], maxlen=200), ('Japan', 'TAK_4502'): deque([], maxlen=60), ('Japan', 'TM_7203'): deque([], maxlen=200), ('Japan', 'SMFG_8316'): deque([], maxlen=200), ('China', 'BGNE_6160'): deque([], maxlen=120), ('China', 'SNP_386'): deque([], maxlen=200)}


In [212]:
def update_z_score():
    global df_global, insight_dict, list_pairs
    for country, adr in list_pairs:
        ticker_ADR, ticker_underlying = adr.split("-")
        lookback_window = diff_record_dict[(country,adr)]
        # initialize lookback window for each stock
        if len(lookback_window) != lookback_window.maxlen:
            adr_price = df_global[f"adr_{ticker_ADR}_close"]
            underlying_price = df_global[f"adr_{ticker_underlying}_close"]
            us_bef_diff = adr_price - underlying_price
            us_aft_diff = (adr_price - underlying_price.shift(-1)).dropna()
            i = 1
            while diff_record_dict[(country,adr)].maxlen != len(diff_record_dict[(country,adr)]):
                lookback_window.appendleft(us_bef_diff.iloc[-i])
                lookback_window.appendleft(us_aft_diff.iloc[-i])
                i += 1
    
        # calculate Z score
        mean = np.array(lookback_window).mean()
        std = np.array(lookback_window).std()
        new_diff = adr_price.iloc[-1] - underlying_price.iloc[-1]
        zscore = ( new_diff - mean)/std
        
        # update lookback window
        lookback_window.append(new_diff)
        
        # make trading decisions
        if (zscore > hp_dict[(country, adr)]["entry"]):
            insight_dict[(country, adr)] = "SHORT"
        elif (zscore < hp_dict[(country, adr)]["exit"]):
            insight_dict[(country, adr)] = "EXIT"
        else: 
            insight_dict[(country, adr)] = "FLAT"
        

Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
[WinError 10054] An existing connection was forcibly closed by the remote host


In [ ]:
def execute_trades():
    global list_pairs
    for (country, adr) in list_pairs:
        
            current = merged_df.loc[(index - temp_risk_lookback + 1):index].copy()
            next_day = merged_df.loc[(index - temp_risk_lookback + 2):(index + 1)].copy()
            stock_values = (np.array((current["stock_close"])/np.array(next_day["avg_non_us_before"]))*stock_quantity) 
            adr_values = np.array(current["adr_close"]*adr_quantity)
            sigma, var, max_drawdown_abs = get_risk_statistics(stock_values, adr_values, var_ci)

In [ ]:
# 8.30 am
    # Pull Historical Forex Data (Minute, for lookback window)
    # Pull Close Data for Asian Market at 9am (Before US Market opens)
# 9.28 am 
    # Pull Forex Data for three currencies (For Z score)
    # Calculate Z Score 
    # for each pair:
        # if invested:
            # if risk condition or exit condition:
                # liquidate
            # else:
                # continue
        # if entry condition:
            # determine order size from risk metric and ADT over last 5 days
            # if 0: no order made
            # else: 
                # Define contract
                # make order (What order? Market Order?)


# 6 pm
    # Pull Close Data for US Market at 6pm (Before Asian Market opens)
    # Pull Historical Forex Data

# 2 min before Asian Market Open
    # Pull Forex Data
    # Calculate Z score using market close and minute forex data

In [127]:
df_temp

,date,open,high,low,close,volume,average,barCount,HKD.USD
0,2021-04-19 17:15:00,7.76573,7.76741,7.76499,7.76592,-1,-1.0,-1,0.128769
1,2021-04-19 17:16:00,7.76575,7.76590,7.76575,7.76590,-1,-1.0,-1,0.128769
2,2021-04-19 17:17:00,7.76575,7.76590,7.76575,7.76590,-1,-1.0,-1,0.128769
3,2021-04-19 17:18:00,7.76575,7.76590,7.76575,7.76590,-1,-1.0,-1,0.128769
4,2021-04-19 17:19:00,7.76574,7.76590,7.76574,7.76590,-1,-1.0,-1,0.128769
5,2021-04-19 17:20:00,7.76574,7.76590,7.76574,7.76590,-1,-1.0,-1,0.128769
6,2021-04-19 17:21:00,7.76574,7.76590,7.76574,7.76590,-1,-1.0,-1,0.128769
7,2021-04-19 17:22:00,7.76574,7.76590,7.76573,7.76590,-1,-1.0,-1,0.128769
8,2021-04-19 17:23:00,7.76573,7.76590,7.76573,7.76590,-1,-1.0,-1,0.128769
9,2021-04-19 17:24:00,7.76573,7.76590,7.76573,7.76590,-1,-1.0,-1,0.128769


In [61]:
df_test = pd.merge(df_forex,df_temp, how = "outer")

In [62]:
df_test

,date,JPY.USD,AUD.USD,HKD.USD,open,high,low,close,volume,average,barCount
0,2020-11-30 17:15:00,0.009587,0.734580,0.129011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-11-30 17:16:00,0.009587,0.734490,0.129011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-11-30 17:17:00,0.009587,0.734445,0.129011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-11-30 17:18:00,0.009587,0.734460,0.129011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-11-30 17:19:00,0.009586,0.734565,0.129011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-11-30 17:20:00,0.009586,0.734610,0.129011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2020-11-30 17:21:00,0.009586,0.734630,0.129011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2020-11-30 17:22:00,0.009586,0.734635,0.129011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2020-11-30 17:23:00,0.009587,0.734695,0.129011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2020-11-30 17:24:00,0.009587,0.734695,0.129011,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [166]:
df_forex = pd.read_csv(f"{path_data}df_forex.csv")
df_forex["date"] = [datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S') for date_time in df_forex["date"]]

In [ ]:
country_info = {
    "Australia":{"Currency": "AUD", 
                "Exchange": "SMART",
                "Pairs": aus_adr_underlying_pairs},
    
    "Japan":{"Currency": "JPY", 
             "Exchange": "TSEJ",
             "Pairs": jpn_adr_underlying_pairs},
    
    "HK":{"Currency": "HKD", 
         "Exchange": "SEHK",
         "Pairs": hk_adr_underlying_pairs},
    
}


live_data = {}
def pull_live_data():
    global live_data, country_info, forex_pairs
    
    dict_forex = {}
    for country in forex_pairs:
        currency1, currency2, * = country

        history = "60 S"
        freq = "1 min"
        side = "BID_ASK"

        forex_contract = Contract(symbol = currency1, secType = "CASH", exchange = "IDEALPRO", currency = currency2)
        ib.qualifyContracts(forex_contract)
        df_temp = get_data(forex_contract, history, freq, side)
        
        latest_tick = df_temp.iloc[-1]
        if currency2 == "USD":
            live_data[f"{currency1}.{currency2}"] = (latest_tick["open"] + latest_tick["close"])/2

        else:
            live_data[f"{currency2}.{currency2}"] = 2/(latest_tick["open"] + latest_tick["close"])
            
    for country in country_info.keys():
        currency = country_info[country]["Currency"]
        exchange = country_info[country]["Exchange"]
        pairs_list = country_info[country]["Pairs"]
        
        side = "Trades"

        for ticker_ADR, ticker_underlying in pairs_list:
            contract = Contract(symbol = ticker_ADR, secType = "STK", exchange = "SMART", currency = currency)
            ib.qualifyContracts(contract)
            df_temp = get_data(contract, history, freq, side)
            latest_tick = df_temp.iloc[-1]
            live_data[ticker_ADR] = latest_tick['close'] # NEED TO MULTIPLY BY ADR RATIO
            
            
            contract = Contract(symbol = ticker_underlying, secType = "STK", exchange = exchange, currency = currency)
            ib.qualifyContracts(contract)
            df_temp = get_data(contract, history, freq, side)
            latest_tick = df_temp.iloc[-1]
            if country == "Australia":
                live_data[ticker_ADR] = latest_tick['close'] * live_data["AUD.USD"] # NEED TO MULTIPLY BY ADR RATIO
            elif country == "Japan":
                live_data[ticker_ADR] = latest_tick['close'] * live_data["JPY.USD"] # NEED TO MULTIPLY BY ADR RATIO
            elif country == "HK":
                live_data[ticker_ADR] = latest_tick['close'] * live_data["HKD.USD"] # NEED TO MULTIPLY BY ADR RATIO
        
    
    
def intraday_trader(HK, AUS, JPN, US):
    if HK:
        pairs = country_info["HK"]["Pairs"]
        for ticker_ADR, ticker_underlying in pairs:
            # check Z Score of pair
            
            if (#z score is exit#){
                
                }
    
    

In [ ]:
initialize_forex()

# update date
while True:
    # keep track of what day it is
    today = datetime.date() # ideally the algorithm starts at 8am in the morning
    
    # keep track of how many times we pull data for calibration
    bef_us_pull = False
    bef_us_zscore = False
    bef_asia_pull = False
    bef_jpn_zscore = False
    bef_aus_zscore = False
    bef_hk_zscore = False
    
    # Various Open and Close hours for various markets
    
    US_open = US_opening(today)
    US_close = US_closing(today)
    HK_open = HK_opening(today + timedelta(days = 1))
    HK_close = HK_closing(today + timedelta(days = 1))
    JPN_open = JPN_opening(today + timedelta(days = 1))
    JPN_close = JPN_closing(today + timedelta(days = 1))
    AUS_open = AUS_opening(today + timedelta(days = 1))
    AUS_close = AUS_closing(today + timedelta(days = 1))
    
    # Keep track if pair is open
    
    while now < US_close and now < HK_close and now < JPN_close and now < AUS_close:
        HK_trading = False
        AUS_trading = False
        US_trading = False
        JPN_trading = False
        '''
        US Market
        '''
        # Before US Market Open pull historical data
        if not bef_us_pull and now > US_open - timedelta(hours = 1):
            bef_us_pull = True
            pull_asian_close()
            pull_forex_data()
            
        
        # Right before US Market Open for Z Score
        if not bef_us_zscore and now > US_open - timedelta(minutes = 4):
            bef_us_zscore = True
            pull_forex_data()
            # Calculate Z score 
            # make trading decision
            convert_price_asian_close()
            
        if US_open < now and now < US_close:
            # Pull minute data to check whether to liquidate positions
            US_trading = True
            pull_live_data()
            intraday_trader(HK_trading, AUS_trading, JPN_trading, US_trading)
            
        if US_close - timedelta(minutes = 10) < now and now < US_close:
            # check pending limit orders and liquidate
                        
        '''
        Asian Markets
        '''
        
        # before asian markets open, pull us market close
        if not bef_asia_pull and now > US_close + timedelta(hours = 1):
            bef_asia_pull = True
            pull_us_close()


        
        # before Hong Kong market open, pull forex to calculate Z score
        if not bef_hk_zscore and now > HK_open - timedelta(minutes = 4):
            bef_hk_zscore = True
            pull_forex_data()
            convert_price_asian_close()
            # calculate Z score and make decision on trade and size
        
        # check on Hong Kong Stocks
        if HK_open < now and now < HK_close:
            HK_trading = True
            
            # Pull minute data to check whether to liquidate positions
            
        if HK_close - timedelta(minutes = 10) < now and now < HK_close:
            # Check pending limit orders and cancel
        
        # before Japan market open, pull forex to calculate Z score
        if not bef_jpn_zscore and now > JPN_opening(today) - timedelta(minutes = 4):
            bef_JPN_zscore = True
            pull_forex_data()
            convert_price_asian_close()
            # calculate Z score
            # make decision on trade and size
        
        # check on Japan stocks
        if JPN_open < now and now < JPN_close:
            # Pull minute data to check whether to liquidate positions
            JPN_trading = True
            
        if JPN_close - timedelta(minutes = 10) < now and now < JPN_close:
            # Check pending limit orders
            
        # before Australia market open, pull forex to calculate Z score            
        if not bef_aus_zscore and now > AUS_opening(today) - timedelta(minutes = 4):
            bef_aus_zscore = True
            pull_forex_data()
            convert_price_asian_close()
        
        # check on Australian Stocks
        if AUS_open < now and now < AUS_close:
            # Pull minute data to check whether to liquidate positions
            AUS_trading = True
            
        if AUS_open - timedelta(minutes = 10) < now and now < AUS_close:
            # Check pending limit orders         

        ib.sleep(120) # Loop every two minutes
        now = datetime.now()

[Contract(secType='STK', conId=13905840, symbol='7751', exchange='TSEJ', primaryExchange='TSEJ', currency='JPY', localSymbol='7751.T', tradingClass='7751')]